In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras.preprocessing import sequence
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Data preview

df_train = pd.read_csv(train_file_path, delimiter='\t', names=['label', 'feature'], header=None)
print(df_train.head(5))

df_test = pd.read_csv(test_file_path, delimiter='\t', names=['label', 'feature'], header=None)


'''with open(train_file_path, 'r') as myfile:
  for line in myfile:
    print(line)'''

In [ ]:
# Preprocessing into data&labels

VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64

train_data, train_labels = df_train['feature'], df_train['label']
test_data, test_labels = df_test['feature'], df_test['label']

#print(train_data[0])


In [ ]:
# integer encoding

vocab = {}
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ")
  encoding = []

  for word in words:
    if word in vocab:
      code = vocab[word]
      encoding.append(code)
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1

  return encoding


train_data_encoded = []
for feature in train_data:
  feature_encoded = one_hot_encoding(feature)
  train_data_encoded.append(feature_encoded)
print(train_data[0])
print(train_data_encoded[0])

test_data_encoded = []
for feature in test_data:
  feature_encoded = one_hot_encoding(feature)
  test_data_encoded.append(feature_encoded)
print(test_data[0])
print(test_data_encoded[0])


train_labels_encoded = np.array([1 if label == 'spam' else 0 for label in train_labels])
test_labels_encoded = np.array([1 if label == 'spam' else 0 for label in test_labels])


In [ ]:
# Preprocessing to have all features same length (MAXLEN)

train_data_encoded_padded = sequence.pad_sequences(train_data_encoded, MAXLEN)
test_data_encoded_padded = sequence.pad_sequences(test_data_encoded, MAXLEN)

In [ ]:
 # Creating the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

 #model.summary()

In [ ]:
# Compile and train the model

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data_encoded_padded, train_labels_encoded, epochs=10)

In [ ]:
# Test the model

results = model.evaluate(test_data_encoded_padded, test_labels_encoded)
print(results)

In [ ]:
# Validate the model

y_pred = np.round(model.predict(test_data_encoded_padded),5).flatten()

df =  pd.DataFrame({'Feature': test_labels, 'Actual': test_labels_encoded, 'Prediction': y_pred})
print(df.tail(30))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred_text_encoded = one_hot_encoding(pred_text)
  pred_text_encoded_padded = sequence.pad_sequences([pred_text_encoded], MAXLEN)
  y_pred = np.round(model.predict(pred_text_encoded_padded), 5).flatten()
  y_pred_label = ['spam' if y_pred >= 0.5 else 'ham']
  prediction = [y_pred[0], y_pred_label[0]]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
